In [1]:
pip install google-cloud-aiplatform


In [15]:
import torch
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer

In [16]:
model_name = "EleutherAI/gpt-neo-125M"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
model.eval()

GPTNeoForCausalLM(
  (transformer): GPTNeoModel(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(2048, 768)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPTNeoBlock(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPTNeoAttention(
          (attention): GPTNeoSelfAttention(
            (attn_dropout): Dropout(p=0.0, inplace=False)
            (resid_dropout): Dropout(p=0.0, inplace=False)
            (k_proj): Linear(in_features=768, out_features=768, bias=False)
            (v_proj): Linear(in_features=768, out_features=768, bias=False)
            (q_proj): Linear(in_features=768, out_features=768, bias=False)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPTNeoMLP(
          (c_fc): Linear(in_features=768, out_features=3072, bias=True)
          (c_proj): Linear(in_fe

In [17]:
if torch.cuda.is_available():
    model.to("cuda")

def generate_text(prompt, max_length=150, temperature=0.7):
    inputs = tokenizer(prompt, return_tensors="pt")
    if torch.cuda.is_available():
        inputs = {k: v.to("cuda") for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=max_length,
            temperature=temperature,
            do_sample=True,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


In [23]:
df = pd.read_csv("/content/IMDB Dataset")
sample_reviews = df.head(5)
sample_reviews

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [36]:
def clean_text(text):
    import re
    text = re.sub(r'<.*?>', ' ', text)
    return ' '.join(text.split())

In [37]:

def basic_prompt(review):
    review = clean_text(review)
    return f"""
What is the sentiment of the following review?
Answer with ONE word only: Positive, Neutral, or Negative.

Review: "{review}"
Sentiment:
"""

In [42]:
def few_shot_prompt(review):
    review = clean_text(review)
    return f"""
You are a sentiment analysis expert. Classify the sentiment of the review with ONE word only: Positive, Neutral, or Negative.

Here are some examples:
- Review: "I love this movie! It was amazing and heartfelt." → Sentiment: Positive
- Review: "The movie was okay, not the best but not the worst." → Sentiment: Neutral
- Review: "I hated this film. It was boring and poorly acted." → Sentiment: Negative

Now, analyze the following review:
- Review: "{review}" → Sentiment:"""


In [43]:
def chain_of_thought_prompt(review):
    review = clean_text(review)
    return f"""
You are a sentiment analyst. Analyze the sentiment of the following review step-by-step.

Review: "{review}"

Instructions:
1. Identify key phrases that express emotion, opinion, or judgment.
2. Classify each phrase as Positive, Negative, or Neutral.
3. Weigh the overall tone of the review.
4. Conclude the overall sentiment with ONE word only: Positive, Neutral, or Negative.

Answer:
"""


In [44]:
def generate_text(prompt, max_new_tokens=100, temperature=0.7):
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024)
    if torch.cuda.is_available():
        inputs = {k: v.to("cuda") for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            do_sample=True,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


In [45]:
print("=== Sentiment Analysis on IMDb Samples ===\n")

for idx, row in sample_reviews.iterrows():
    review_text = row['review']
    print(f"Review #{idx+1}: {review_text}\n")
    print("Basic Prompt Output:")
    print(generate_text(basic_prompt(review_text)), "\n")
    print("Few-Shot Prompt Output:")
    print(generate_text(few_shot_prompt(review_text)))
    print("Chain-of-Thought Prompt Output:")
    print(generate_text(chain_of_thought_prompt(review_text)), "\n")

    print("="*2 + "\n")

=== Sentiment Analysis on IMDb Samples ===

Review #1: One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I 